In [1]:
!pip install pinecone-client wikipedia google-api-python-client unstructured tabulate pdf2image

In [22]:
from langchain.document_loaders import DirectoryLoader

loader = DirectoryLoader(
    './plots', # my local directory
    glob='**/*.pdf',     # we only get pdfs
)
docs = loader.load()
docs

[Document(page_content='On Earth-65 in New York City, over a year after helping defeat Kingpin with Miles Morales, Gwen Stacy relates her struggle to create relationships with anyone around her, feeling isolated in her role as Spider-Woman. After quitting her band, she remembers the last person she felt close to, her best friend Peter Parker. He, along with his Aunt May, was very close to Gwen and her father, Police Captain George Stacy, though she never told any of them about her double life as a superhero. Peter was bullied relentlessly at school, despite Gwen trying to defend him. Unbeknownst to her, he started experimenting with a dangerous chemical. The night of prom, a giant lizard monster attacked the school and tried to kill Peter\'s bully. Gwen, leaping into action as Spider-Woman, fought him off, causing the building to collapse on top of the creature. The lizard reverted to Peter, who was mortally injured by the rubble. Gwen tried to save him, but it was too late. Peter reve

In [23]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=0
)
docs_split = text_splitter.split_documents(docs)
docs_split

Created a chunk of size 2818, which is longer than the specified 1000
Created a chunk of size 1173, which is longer than the specified 1000
Created a chunk of size 1261, which is longer than the specified 1000
Created a chunk of size 2005, which is longer than the specified 1000
Created a chunk of size 2067, which is longer than the specified 1000
Created a chunk of size 2265, which is longer than the specified 1000
Created a chunk of size 1154, which is longer than the specified 1000
Created a chunk of size 1931, which is longer than the specified 1000
Created a chunk of size 1422, which is longer than the specified 1000


[Document(page_content='On Earth-65 in New York City, over a year after helping defeat Kingpin with Miles Morales, Gwen Stacy relates her struggle to create relationships with anyone around her, feeling isolated in her role as Spider-Woman. After quitting her band, she remembers the last person she felt close to, her best friend Peter Parker. He, along with his Aunt May, was very close to Gwen and her father, Police Captain George Stacy, though she never told any of them about her double life as a superhero. Peter was bullied relentlessly at school, despite Gwen trying to defend him. Unbeknownst to her, he started experimenting with a dangerous chemical. The night of prom, a giant lizard monster attacked the school and tried to kill Peter\'s bully. Gwen, leaping into action as Spider-Woman, fought him off, causing the building to collapse on top of the creature. The lizard reverted to Peter, who was mortally injured by the rubble. Gwen tried to save him, but it was too late. Peter reve

In [24]:
import os
import pinecone 

PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]
PINECONE_ENV = os.environ["PINECONE_ENV"]
(os.environ["PINECONE_API_KEY"], os.environ["PINECONE_ENV"])

# pinecone.delete_index(index_name)
# pinecone.create_index(index_name, dimension=1536)

('c7da49b3-0ac9-404d-889a-6a5ce3b1ba74', 'us-west4-gcp-free')

In [25]:
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings

# we use the openAI embedding model
embeddings = OpenAIEmbeddings()
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENV
)

index_name = "langchain-demo"

doc_db = Pinecone.from_documents(
    docs_split, 
    embeddings, 
    index_name=index_name
)

In [26]:
query = "Who is Spot?"
search_docs = doc_db.similarity_search(query)
search_docs

[Document(page_content='in ruins of the Alchemax collider. The Spot tells Spider-Man that this is where they were both created, as Spot used to be a scientist at Alchemax (the one who Miles threw a bagel at in the ﬁrst ﬁlm), and ran the test which brought the spider that bit Miles through from another dimension. Likewise, when Miles destroyed the collider, the energy released caused an accident that turned Spot permanently into the humanoid portal creature his is now. The Spot was shunned by his colleagues, friends, and even family for his condition. He becomes enraged, and tries to attack Spider-Man and Lieutenant Morales, but accidentally kicks himself on his butt and falls into a deep portal, which closes behind him. Jeff is frustrated that the criminal got away, as well as all the property damage caused by the ﬁght. He conﬁdes in Spider-Man about his struggles to be a good father and husband, and the masked Miles awkwardly offers advice before leaving. The Spot ﬁnds himself in a vo

In [27]:
from langchain import OpenAI
llm = OpenAI()

In [33]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type='stuff',
    retriever=doc_db.as_retriever(),
)

query = "Who is Spot?"
result = qa.run(query)

result

' Spot is a small time villain who was formerly a scientist at Alchemax. He was transformed into a humanoid portal creature after an accident at the Alchemax collider. He has the power to create portals and travel to different dimensions.'

In [30]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

memory = ConversationBufferMemory(memory_key='chat_history',return_messages=True)
qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0), doc_db.as_retriever(), memory=memory)

query = "What happened to Spot?"
result = qa.run(query)
print(result)

query = "How did he die?"
result = qa.run(query)
print(result)

 Spot escaped through a portal and found himself in a void, where he saw many portals made by his spots. He determined to find a way to get more power to control the void and defeat Spider-Man. Several months later, he was unsuccessfully trying to rob an ATM using his spots. Miles Morales eventually subdued him, but he escaped again. He eventually gained a great deal more control over his powers and was able to activate a collider, absorbing its power. He then escaped through a portal, vowing revenge on Spider-Man.
 Spot did not die. He escaped through a portal and was last seen in a void, where he saw many portals made by his spots. He determined to find a way to get more power to control the void and defeat Spider-Man.
